In [2]:
###########################
# 7-2 Project Two Submission: Complete Dashboard & Visualizations
# Modified by: Michael Murphy
# Date: 2025 10-04
# Dashbaord for the Grazioso Salvare rescue animal organization
# Dashboard with filterable columns, filter buttons, breed/outcome pie chart, and geolocation map
###########################

# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports
import dash_leaflet as dl                   # Dash Leaflet for interactive maps
from dash import dcc, html                  # Dash Core Components for dashboard widgets, and HTML for layout 
import plotly.express as px                 # Plotly Express for creating charts (pie, line, bar, etc.)
from dash import dash_table                 # Dash DataTable for displaying and interacting with tabular data
from dash.dependencies import Input, Output # Used to define callback inputs and outputs
from dash import ctx                        # Context object to identify which input triggered a callback
from dash import no_update                  # Used to prevent updating a component when no change is needed

import base64
JupyterDash.infer_jupyter_proxy_config()
#from pymongo import MongoClient, errors    # Error handling, used to identify login errors (not in use, hard coded login)

# Configure Operating system interfaces (file paths, environment variables, directory operations, etc.)
# import os   # Not in use

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# import for your CRUD module
from CRUD_Python_Module_7 import AnimalShelter
### NOTE: I am now naming my CRUD by course module to allow comparison to previous versions.
### Instructor testing this module may require renaming the module


###########################
# Data Manipulation / Model
###########################
username = "aacuser"
password = "password"
host = "127.0.0.1"
port = 27017
database = "aac"
collection = "animals"

# Animal Shelter Connection
db = AnimalShelter(username, password, host, port, database, collection)

# class read method returns of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invalid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will return a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
#print(len(df.to_dict(orient='records')))
#print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    
    # Header   
    html.Div([

        html.A(
            html.Img(   # Grazioso Salvare Logo
                src='/files/code_files/Grazioso Salvare Logo.png',
                style={'width': '60px', 'height': 'auto', 'margin-right': '10px'}
            ),
            href="https://www.snhu.edu",
            target="_blank"  # Opens in a new tab
        ),
        html.H1('SNHU CS-340 Grazioso Salvare Rescue Dog Dashboard', style={'margin': 0}),   # Page Title
        html.Img(   # Unique identifier using my account icon
            src='/files/code_files/MichaelMurphy_BlueBurst.png',
            style={'width': '50px', 'height': 'auto', 'margin-right': '10px'}
        ),        
    ], style={'display': 'flex', 'align-items': 'center', 'justify-content': 'center'}),

    # Spacing
    html.Hr(),

    # Filters: Buttons & Dropdowns
    html.Div(className='buttonRow', 
            style={'display': 'flex', 'gap': '10px'},  # add spacing between items
                children=[
                    # define 2 buttons for quick filtering
                    html.Button(id='submit-button-reset', n_clicks=0, children='Reset Display'),
                    html.Button(id='submit-button-dogs', n_clicks=0, children='All Dogs'),

                    # Rescue type selection dropdown
                    dcc.Dropdown(
                        id='rescue-dropdown',
                        options=[
                            {'label': 'Disaster / Individual Tracking', 'value': 'disaster'},
                            {'label': 'Mountain or Wilderness Rescue', 'value': 'mountain'},
                            {'label': 'Water Rescue', 'value': 'water'}
                        ],
                        value=None,  # no default selection
                        placeholder="Rescue Type",  # shows this text/title before selection
                        style={'width': '75%'}
                    ),

                    # Pie Chart selection dropdown
                    dcc.Dropdown(
                        id='chart-dropdown',
                        options=[
                            {'label': 'Breed', 'value': 'breed'},
                            {'label': 'Outcome Type', 'value': 'outcome_type'}
                        ],
                        value=None,  # no default selection
                        placeholder="Pie Chart: Breed / Outcome",  # shows this text/title before selection
                        style={'width': '75%'}
                    )                  
                ]
    ),    
    
    # Table
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True}
                                  for i in df.columns],
                         data=df.to_dict('records'),
                         editable=False,
                         filter_action="native",
                         sort_action="native",
                         sort_mode="multi",
                         column_selectable=False,
                         #row_selectable=False,
                         row_selectable="single",   # Set to allow only sing row selection
                         row_deletable=False,
                         selected_columns=[],
                         selected_rows=[0],   # Default selected row to 0
                         page_action="native",
                         page_current=0,
                         page_size=10),

    # Spacing
    html.Br(),
    html.Hr(),
    
    # Display for the pie chart & geolocation map, in a side-by-side configuration
    html.Div(className='row',
        style={'display' : 'flex'},
        children=[
            # Pie Chart
            dcc.Graph(id="graph-id"),

            # geolocation map
            html.Div(id='map-id',
                className='col s12 m6')
        ]
    )

])   # end of app.layout

#############################################
# Interaction Between Components / Controller
#############################################

# This callback will handle button clicks & dropdown for filtering data
@app.callback(
    [Output('datatable-id', 'data'),           # Update the data table
     Output('rescue-dropdown', 'value'),       # update the rescue dropdown (for resetting)
     Output('chart-dropdown', 'value'),        # update the breed/outcome piechrt dropdown (for resetting)
     Output('datatable-id', 'filter_query'),   # update the tabel's filter (for resetting)
    Output('datatable-id', 'selected_rows')],   # update the tabel's selected row (for resetting)
    [Input('submit-button-dogs', 'n_clicks'),
     Input('submit-button-reset', 'n_clicks'),
     Input('rescue-dropdown', 'value')],
    prevent_initial_call=True
)

def on_click(button_click_dogs, button_click_reset, rescue_dropdown_value):
    triggered_id = ctx.triggered_id

    # As a default, the table filter and chart / rescue dropdowns states will not be modified/reset
    chart_dropdown_result = no_update 
    rescue_dropdown_result = no_update
    table_filter_result = no_update
    table_selected_row = no_update
    
    #Set up querys for each rescue type
    disaster_rescue_query = {
        "animal_type": "Dog",
        "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", 
                          "Bloodhound", "Rottweiler"]},           # any of these breeds
        "sex_upon_outcome": "Intact Male",                        # exact match
        "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300},   # numeric range, inclusive
        "outcome_type": {"$nin": ["Euthanasia", "Died"]}          # exclude deceaseed animals
    }

    mountain_rescue_query = {
        "animal_type": "Dog",
        "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog",
                          "Siberian Husky", "Rottweiler"]},       # any of these breeds
        "sex_upon_outcome": "Intact Male",                        # exact match
        "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156},   # numeric range, inclusive
        "outcome_type": {"$nin": ["Euthanasia", "Died"]}          # exclude deceaseed animals
    }

    water_rescue_query = {
        "animal_type": "Dog",
        "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},  # any of these breeds
        "sex_upon_outcome": "Intact Female",                      # exact match
        "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156},   # numeric range, inclusive
        "outcome_type": {"$nin": ["Euthanasia", "Died"]}          # exclude deceaseed animals
    }
    
    # Button Selections
    if triggered_id == "submit-button-dogs":   # Select All Dogs
        df = pd.DataFrame.from_records(db.read({"animal_type": "Dog"}))
        rescue_dropdown_result = None   # reset rescue type dropdown
        chart_dropdown_result = None
        table_filter_result = ''        # reset any user-set text filter

    #Dropdown selections
    elif triggered_id == "rescue-dropdown" and rescue_dropdown_value == 'disaster':
        df = pd.DataFrame.from_records(db.read(disaster_rescue_query))

    elif triggered_id == "rescue-dropdown" and rescue_dropdown_value == 'mountain':
        df = pd.DataFrame.from_records(db.read(mountain_rescue_query))    
        
    elif triggered_id == "rescue-dropdown" and rescue_dropdown_value == 'water':
        df = pd.DataFrame.from_records(db.read(water_rescue_query))   

    # Default is to Reset the page to initial state (Clicking the Reset button)
    else:
        df = pd.DataFrame.from_records(db.read({}))
        rescue_dropdown_result = None   # When the reset button is clicked, reset the dropdows back to their unset states
        chart_dropdown_result = None
        table_filter_result = ''        # reset any user-set text filter
        table_selected_row = [0]        # reset the selected row to starting row
    
    # Cleanup Mongo _id field
    df.drop(columns=['_id'], inplace=True, errors='ignore')
    # return the table records, and reset the dropdowns and text filter, if needed
    return df.to_dict('records'), rescue_dropdown_result, chart_dropdown_result, table_filter_result, table_selected_row


# This callback will highlight a row on the data table when the user selects it
# corrected code to highlight rows correctly, instead of unsuccessfully highlighting columns
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_rows')],
    prevent_initial_call=True
)

def update_styles(selected_rows):
    if not selected_rows:
        return []

    return [
        {
            'if': {'row_index': r}, 
            'backgroundColor': '#D2F3FF'
        }
        for r in selected_rows
    ]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")],
    prevent_initial_call=True
)

# geolocation chart
def update_map(viewData, index):
    # exit if there is no viewData
    if viewData is None:
        return [html.P("No animals found for this filter.")]

    dff = pd.DataFrame.from_dict(viewData)
    
    # Prevent traceback error for having no data, and inform the user of needed selection
    if dff.empty:
        return [html.P("No animals found for this filter.")]
    
    # validate the index is set
    if not index:        # None or empty list
        row = 0          # Set row directly to 0, which is the initial row
    # Because we only allow single row selection, the list can 
    # be converted to a row index here
    else: 
        row = index[0]   # set row to the first selected row (there should only be 1 row anyway)
      
    # GeoLocation Map is created and returned for display
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'},
           # Austin TX is at [30.75,-97.48], so that is where we center the map
           center=[30.75,-97.48], zoom=10, children=[
           dl.TileLayer(id="base-layer-id"),
           # Marker with tool tip and popup
           # Column 13 and 14 define the grid-coordinates for the map
           # Column 4 defines the breed for the animal
           # Column 9 defines the name of the animal
           dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]],
              children=[
              dl.Tooltip(dff.iloc[row,4]),
              dl.Popup([
                html.H3("Animal Name:"),
                html.H3(dff.iloc[row,9])
             ])
          ])
       ])
    ]



# This callback is for the Pie Chart

@app.callback(
    Output("graph-id", "figure"),
    [Input("chart-dropdown", "value"),
     Input("datatable-id", "derived_virtual_data")]
)

def generate_chart(selected_column, rows):
    dff = pd.DataFrame(rows)

    if dff.empty or selected_column not in dff:
        return px.pie(title="No data: Use the dropdown to select Breed or Outcome Type")

    # Count frequencies for the selected column
    counts = dff[selected_column].value_counts().reset_index()
    counts.columns = [selected_column, "count"]

    fig = px.pie(
        counts,
        values="count",
        names=selected_column,
        hole=0.3,
        title=f"Distribution of {selected_column.capitalize()}"
    )

    # Show label and count on the chart, show % on hover
    fig.update_traces(
        textinfo="label+value",  
        hovertemplate="<b>%{label}</b><br>Count: %{value}<br>Percent: %{percent}<extra></extra>"
    )    
    
    return fig




# Run app and display result in jupyterlab mode, note, if you have previously run a prior app, the default port of 8050 may not be available, if so, try setting an alternate port.
#app.run_server()
app.run_server(mode='jupyterlab', port=8050, debug=True)

In [1]:
# Test Suite for CRUD_Python_Module.py
# CS-340, Rescue Animal Database Project
# Author: Michael Murphy
# Date: 2025 09-14

# This cell and the cells below are the test suite for the dashboard environment and the CRUD module
# You only need to run these if you want to perform testing


# Simple print and reload test to confirm Jupyter lab functionality
# If this doesn't work, restart the kernal.

print("hello world")

%load_ext autoreload
%autoreload 2

import sys
print(sys.executable)


import socket

sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
result = sock.connect_ex(('127.0.0.1', 8054))
if result == 0:
    print("Port 8054 is in use")
else:
    print("Port 8054 is free")
sock.close()

#app.run_server(port=8054, debug=True)

print("Kernel is alive")

hello world
/home/codio/.pyenv/versions/3.11.2/bin/python3.11
Port 8054 is free
Kernel is alive


In [ ]:
# Perform Import of database CRUD module
# This must be run before any other of the test cells below.
from CRUD_Python_Module_7 import AnimalShelter

print("Import Ran")

In [ ]:
# TEST: Create Record in Database 
#    Test Steps:
# 1) build an animal record
# 2) Connect to Database
# 3) Call create() to add record
# 4) Display Fuccess/Failure


# New record to insert (MongoDB generates the _Id automatically, so it is not included)
new_animal = {
    "rec_num": 1,              # This is expected to be ignored and updated based on current max record number
    "age_upon_outcome": "2 years",
    "animal_id": "A000000",
    "animal_type": "Example",  # This animal_type can be searched for on the Dashboard to verify it exists after it is inserted
    "breed": "Example Breed",
    "color": "Rainbow",
    "date_of_birth": "2055-10-10",
    "datetime": "2015-12-28 18:43:00",
    "monthyear": "2015-12-28T18:43:00",
    "name": "Example Name",
    "outcome_subtype": "",
    "outcome_type": "Adoption",
    "sex_upon_outcome": "Neutered Male",
    "location_lat": 10.7595748121648,
    "location_long": -10.5523753807133,
    "age_upon_outcome_in_weeks": 110.111408730159
}

# Animal Shelter DB connection Info
username = "aacuser"
password = "password"
host = "127.0.0.1"
port = 27017
database = "aac"
collection = "animals"

# Connect to the Database
db = AnimalShelter(username, password, host, port, database, collection)

# Insert the record
success = db.create(new_animal)
print("Creation Sucess?:", success)


In [ ]:
# TEST: Update Records
# Update a set of records that match the query value
#    Test Steps:
# 1) Connect to Database
# 2) Build a query that matches the animal record from the create test
# 3) Build a set of changed fields to test updating multipel fields at once
# 4) Call update() to add change all matching records
# 4) Display number of records updated

# Animal Shelter DB connection Info
username = "aacuser"
password = "password"
host = "127.0.0.1"
port = 27017
database = "aac"
collection = "animals"

# Connect to the Database
db = AnimalShelter(username, password, host, port, database, collection)

# Build a query to find (and modify) all the Example records we created
query = {"animal_type": "Example"}
changed_fields = {"name": "Updated Example Name", "outcome_type": "Updated Outcome"}

# Update Example Records
updated_number = db.update(query, changed_fields)
print("Number of records updated:", updated_number)


In [ ]:
# TEST: Delete Records
# Remove all Example Records from Database
# This test cleans up the test environment; we can then omit the create test and verify the DB does NOT have the Example record
#    Test Steps:
# 1) Connect to Database
# 2) Build a query that matches the animal record from the create test
# 3) Call delete() to remove all matching records
# 4) Display number of records deleted


# Animal Shelter DB connection Info
username = "aacuser"
password = "password"
host = "127.0.0.1"
port = 27017
database = "aac"
collection = "animals"

# Connect to the Database
db = AnimalShelter(username, password, host, port, database, collection)

# Build a query to find (and delete) all the Example records we created
query = {"animal_type": "Example"}

# Delete Example Records
deletion_number = db.delete(query)
print("Number of records delteted:", deletion_number)
